In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization
from sklearn.utils.class_weight import compute_class_weight

In [8]:
# Set your data directories
# data_dir = '../Gait Analysis/'
img_height = 64
img_width = 64
batch_size = 32
epochs = 5

# Create a label mapping
class_labels = {'normal': 0, 'abnormal': 1}

# Load images and labels
dirlist=[abnormal_dir,healthy_dir ]
classes=['Yes', 'No']
filepaths=[]
labels=[]
for i,j in zip(dirlist, classes):
    filelist=os.listdir(i)
    for f in filelist:
        filepath=os.path.join (i,f)
        filepaths.append(filepath)
        labels.append(j)
print ('filepaths: ', len(filepaths), '   labels: ', len(labels))

# Convert to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize pixel values to [0, 1]
images = images / 255.0

filepaths:  23560    labels:  23560


In [9]:
# Count the number of images for each label
num_normal_images = np.sum(labels == class_labels['normal'])
num_abnormal_images = np.sum(labels == class_labels['abnormal'])

print("Number of 'normal' images:", num_normal_images)
print("Number of 'abnormal' images:", num_abnormal_images)


Number of 'normal' images: 0
Number of 'abnormal' images: 0


C:\Users\ASUS\AppData\Local\Temp\ipykernel_6656\2972066377.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  num_normal_images = np.sum(labels == class_labels['normal'])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_6656\2972066377.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  num_abnormal_images = np.sum(labels == class_labels['abnormal'])


In [12]:
Fseries = pd.Series(X, name="filepaths",dtype=str)
Lseries = pd.Series(y, name="labels",dtype=str)
tumor_data = pd.concat([Fseries,Lseries], axis=1)
tumor_df = pd.DataFrame(tumor_data)

print(tumor_df.head())
print("---------------------")
print(tumor_df["labels"].value_counts())

NameError: name 'pd' is not defined

In [11]:

#splitting data
from sklearn.model_selection import train_test_split
train_images, test_images = train_test_split(tumor_df, test_size=0.3, random_state=42)
train_set, val_set = train_test_split(tumor_df, test_size=0.2, random_state=42)


NameError: name 'tumor_df' is not defined

In [10]:
# Reshape your image data to have one timestep per row
x_train = x_train.reshape(x_train.shape[0], img_height, img_width * 3)
x_val = x_val.reshape(x_val.shape[0], img_height, img_width * 3)
x_test = x_test.reshape(x_test.shape[0], img_height, img_width * 3)

In [11]:

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

In [13]:
model = Sequential()
model.add(LSTM(256, input_shape=(img_height, img_width * 3), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Second LSTM layer
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Third LSTM layer
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with class weights
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with validation data and early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    x_train, y_train, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=(x_val, y_val),  
    callbacks=[early_stopping],
    class_weight=class_weight_dict  # Pass class weights here
)


Epoch 1/10
589/589 [==============================] - 530s 893ms/step - loss: 0.7441 - accuracy: 0.5172 - val_loss: 0.6912 - val_accuracy: 0.5446
Epoch 2/10
589/589 [==============================] - 422s 716ms/step - loss: 0.6944 - accuracy: 0.5348 - val_loss: 0.6824 - val_accuracy: 0.5739
Epoch 3/10
589/589 [==============================] - 417s 707ms/step - loss: 0.6855 - accuracy: 0.5647 - val_loss: 0.6874 - val_accuracy: 0.5216
Epoch 4/10
589/589 [==============================] - 630s 1s/step - loss: 0.6886 - accuracy: 0.5345 - val_loss: 0.6932 - val_accuracy: 0.4053
Epoch 5/10
589/589 [==============================] - 497s 843ms/step - loss: 0.6910 - accuracy: 0.5413 - val_loss: 0.6814 - val_accuracy: 0.6057
Epoch 6/10
589/589 [==============================] - 670s 1s/step - loss: 0.6685 - accuracy: 0.6040 - val_loss: 0.6727 - val_accuracy: 0.6184
Epoch 7/10
589/589 [==============================] - 497s 843ms/step - loss: 0.5299 - accuracy: 0.7512 - val_loss: 0.4542 - val_a

In [14]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print the test loss and accuracy
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

74/74 [==============================] - 16s 219ms/step - loss: 0.2196 - accuracy: 0.9189
Test Loss: 0.21961523592472076
Test Accuracy: 0.9189304113388062
